In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from IPython.display import Markdown

from src.display import ModelView
from src.stats import LinearRegression, add_constant
from src.test_ import FTest, TTest

In [3]:
columns = {
    'Сводные данные о продаже наличной иностранной '
    'валюты кредитными организациями физическим лицам (x1)': 'x1',
    'Курс рубля к доллару США (x2)': 'x2',
    'Цена нефти марки "Юралс" (x3)': 'x3',
    'Динамика потребительских цен по группам товаров и услуг '
    '(месяц к соответствующему месяцу предыдущего года,%)  (x4)': 'x4',
    'Объем денежных накоплений в РФ миллиард руб. (y)': 'y'
}

In [4]:
df = pd.read_excel('data/task.xlsx', sheet_name='Лист1', header=3, usecols=[1, 2, 3, 4, 5])
df = df.rename(columns=columns)

In [5]:
y = df['y']
multiple_params = ['x1', 'x2', 'x3', 'x4']
x = add_constant(df[multiple_params])

In [6]:
model = LinearRegression(y, x)
model_view = ModelView(model)

In [7]:
Markdown('<br>'.join([
    model_view.specification(inline=True),
    model_view.true_form(inline=True)
]))

$ y_i = a_{0} + b_{1} x_{i1} + b_{2} x_{i2} + b_{3} x_{i3} + b_{4} x_{i4} + e_i $<br>$ \hat{y_i} = \hat{a_{0}} + \hat{b_{1}} x_{i1} + \hat{b_{2}} x_{i2} + \hat{b_{3}} x_{i3} + \hat{b_{4}} x_{i4} $

In [8]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     17.70
Date:                Thu, 07 Oct 2021   Prob (F-statistic):           6.83e-06
Time:                        10:11:11   Log-Likelihood:                -168.61
No. Observations:                  22   AIC:                             347.2
Df Residuals:                      17   BIC:                             352.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.04e+04   6169.698      1.686      0.1

In [9]:
Markdown(model_view.parameterized())

$$ \hat{y_i} = 1.04e+04 + 0.484 x_{i1} + 130 x_{i2} + -6.47 x_{i3} + -276 x_{i4} $$

In [10]:
Markdown('<br>'.join(model_view.bse(inline=True)))

$ S_\hat{a} = 6.17e+03 $<br>$ S_{\hat{b_{1}}} = 0.196 $<br>$ S_{\hat{b_{2}}} = 132 $<br>$ S_{\hat{b_{3}}} = 25.7 $<br>$ S_{\hat{b_{4}}} = 78.9 $

In [11]:
Markdown(model_view.rmsd_resid())

$$ S_{e_i} = 586 $$

In [12]:
Markdown(model_view.rsquared())

$$ R^2 = 0.806 $$

In [13]:
Markdown(model_view.mapprxe())

$$ A = 2.6 $$

## Оценка модели на значимость

In [14]:
f_test = FTest(model)
Markdown(f_test.report())

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 17.7 $<br>
$ p\text{-}value_F = 6.83e-06 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.21 $
<br><br>
$ 6.83e-06 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [15]:
t_test = TTest(model)
Markdown(t_test.report())

### t-критерий Стьюдента.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.11 $
<br>

<br>
$ t_{a} = 1.69 $, $ p\text{-}value_{t_{a}} = 0.11 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{1}} = 2.46 $, $ p\text{-}value_{t_{b_{1}}} = 0.0247 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{2}} = 0.986 $, $ p\text{-}value_{t_{b_{2}}} = 0.338 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{3}} = -0.252 $, $ p\text{-}value_{t_{b_{3}}} = 0.804 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{4}} = -3.5 $, $ p\text{-}value_{t_{b_{4}}} = 0.00274 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.